In [1]:
print("hello world")

hello world


In [2]:
import numpy as np

In [3]:
import matplotlib as mpl

In [4]:
import matplotlib.pyplot as plt
img = plt.imread("./../images/mnist/train_0.bmp")
plt.imshow(img)

In [5]:
from pynq import Overlay

In [6]:
dma_test_design = Overlay("./../bitstreams/dma_test.bit")

In [7]:
dma_test_design.ip_dict

{'axi_dma_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xaf6f5a70>,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S_AXI_LITE',
  'parameters': {'C_BASEADDR': '0x40400000',
   'C_DLYTMR_RESOLUTION': '125',
   'C_ENABLE_MULTI_CHANNEL': '0',
   'C_FAMILY': 'zynq',
   'C_HIGHADDR': '0x4040FFFF',
   'C_INCLUDE_MM2S': '1',
   'C_INCLUDE_MM2S_DRE': '0',
   'C_INCLUDE_MM2S_SF': '1',
   'C_INCLUDE_S2MM': '1',
   'C_INCLUDE_S2MM_DRE': '0',
   'C_INCLUDE_S2MM_SF': '1',
   'C_INCLUDE_SG': '0',
   'C_INCREASE_THROUGHPUT': '0',
   'C_MICRO_DMA': '0',
   'C_MM2S_BURST_SIZE': '16',
   'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
   'C_M_AXIS_MM2S_TDATA_WIDTH': '32',
   'C_M_AXI_MM2S_ADDR_WIDTH': '32',
   'C_M_AXI_MM2S_DATA_WIDTH': '64',
   'C_M_AXI_S2MM_ADDR_WIDTH': '32',
   'C_M_AXI_S2MM_DATA_WIDTH': '32',
   'C_M_AXI_SG_ADDR_WIDTH': '32',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_NUM_MM2S_CHANNELS': '1',
   'C_NUM_S2MM

In [8]:
hex(dma_test_design.ip_dict["axi_gpio_0"]["phys_addr"])

'0x41200000'

In [10]:
from pynq.lib import AxiGPIO

buttons_instance = dma_test_design.ip_dict['axi_gpio_0']
buttons = AxiGPIO(buttons_instance).channel1

In [11]:
buttons.read()

0

In [12]:
switches_instance = dma_test_design.ip_dict['axi_gpio_2']
switches = AxiGPIO(switches_instance).channel1

In [13]:
switches.read()

2

In [23]:
led_instance = dma_test_design.ip_dict['axi_gpio_1']
led = AxiGPIO(led_instance).channel1

In [24]:
led[0:4].write(0x1)

In [25]:
from time import sleep

while(1):
    led[0:4].write(0x1)
    sleep(1)
    led[0:4].write(0x3)
    sleep(1)
    led[0:4].write(0x7)
    sleep(1)
    led[0:4].write(0xf)
    sleep(1)
    led[0:4].off()
    sleep(1)

KeyboardInterrupt: 

In [ ]:
led[0:4].off()

In [9]:
dma_send_recv = dma_test_design.axi_dma_0

In [24]:
import pynq.lib.dma
from pynq import allocate
import time

buf_size = 2621440 #1024*1024/4
input_buf = allocate(shape=(buf_size,), dtype=np.uint32)
output_buf = allocate(shape=(buf_size,), dtype=np.uint32)
for i in range(buf_size):
    input_buf[i] = i

In [25]:
start = time.time()
dma_send_recv.sendchannel.transfer(input_buf)
dma_send_recv.recvchannel.transfer(output_buf)
dma_send_recv.sendchannel.wait()
dma_send_recv.recvchannel.wait()
end = time.time()
print('time cost: ' + str(round(end-start,5)) + ' s')

time cost: 0.04762 s


In [26]:
for i in range(32):
    print(str(output_buf[i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [27]:
len = buf_size*4
cost = round(end-start,5)
speed = round(len/(cost*len),5)
print('transfer speed: ' + str(speed) + ' MB/s')

transfer speed: 20.99958 MB/s
